In [134]:
import pandas as pd
import numpy as np
import surprise
from sklearn.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
from surprise import Reader, Dataset, accuracy, SVD

In [147]:
# 데이터 로드 및 전처리
df = pd.read_csv('../df.csv')
df.reset_index(inplace=True, drop=True)
df = df[['user_id', 'movie_id', 'rating']]
df = df[:50000]
train, test = train_test_split(df, test_size= 0.2, random_state= 42)

reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(df, reader)
traindata = Dataset.load_from_df(train, reader)
testdata = Dataset.load_from_df(test, reader)

SVD

In [154]:
param_grid = {'n_factors' : [70, 100, 130], 'n_epochs' : [10, 20, 40], 'reg_all' : [0.05, 0.1],
              'lr_all' : [0.01, 0.05, 0.5]} #best params > n_ecpochs = 10, lr_allkiuy = 0.01
grid_search = GridSearchCV(SVD, param_grid, measures= ['RMSE'], cv= 5, n_jobs= -1)
grid_search.fit(traindata)

print(grid_search.best_params['rmse'])

{'n_factors': 70, 'n_epochs': 10, 'reg_all': 0.05, 'lr_all': 0.01}


In [152]:
traindata = Dataset.load_from_df(train, reader)
best_algo = grid_search.best_estimator['rmse']
best_algo.fit(traindata)

testset = testdata.build_full_trainset()
# testset_test = testset_full.build_testset()
predictions = best_algo.test(testset)

# RMSE 출력
rmse = accuracy.rmse(predictions)
print(f'Test RMSE : {rmse}')

AttributeError: 'DatasetAutoFolds' object has no attribute 'n_users'

In [132]:
user_rated_movies = df[df['user_id']== 2]['movie_id'].tolist()
all_movies = df['movie_id'].unique()
# all_movies
unrated_movies = [movie for movie in all_movies if movie not in user_rated_movies]
# unrated_movies

predictions = [best_algo.predict(2, movie) for movie in unrated_movies]
predictions.sort(key = lambda x : x.est, reverse = True) # 예상 평점 기준 정렬

top_n_recommendations = predictions[:10]
top_n_recommendations

list = [(pred.iid, pred.est) for pred in top_n_recommendations]
print(list)

[(10002, 10), (100022, 10), (10004, 10), (10005, 10), (10006, 10), (10016, 10), (10018, 10), (10020, 10), (10028, 10), (10035, 10)]


In [141]:
# 특정 사용자에 대한 영화 추천 함수
def get_recommendations(algo, user_id, df, n=20):
    # 사용자가 평가하지 않은 영화 목록 생성
    user_rated_movies = df[df['user_id'] == user_id]['movie_id'].tolist()
    all_movies = df['movie_id'].unique()
    unrated_movies = [movie for movie in all_movies if movie not in user_rated_movies]
    
    # 예상 평점 계산
    predictions = [algo.predict(user_id, movie) for movie in unrated_movies]
    predictions.sort(key=lambda x: x.est, reverse=True) #예상 평점 기준 정렬
    
    # 상위 n개의 영화 추천
    top_n_recommendations = predictions[:n]
    return [(pred.iid, pred.est, pred.r_ui, pred.details) for pred in top_n_recommendations] # (영화id, 예상 평점)

In [145]:
# 사용자 ID에 대한 추천 영화 목록 출력
user_id = 2  
recommended_movies = get_recommendations(best_algo, user_id, df, n=20)
print("추천된 영화 목록:")
for movie_id, est_rating, real_ratings, details in recommended_movies:
    print(f"영화 ID: {movie_id}, 예상 평점: {est_rating}")

추천된 영화 목록:
영화 ID: 10002, 예상 평점: 10
영화 ID: 100022, 예상 평점: 10
영화 ID: 10004, 예상 평점: 10
영화 ID: 10005, 예상 평점: 10
영화 ID: 10006, 예상 평점: 10
영화 ID: 10016, 예상 평점: 10
영화 ID: 10018, 예상 평점: 10
영화 ID: 10020, 예상 평점: 10
영화 ID: 10028, 예상 평점: 10
영화 ID: 10035, 예상 평점: 10
영화 ID: 10038, 예상 평점: 10
영화 ID: 100399, 예상 평점: 10
영화 ID: 10047, 예상 평점: 10
영화 ID: 10048, 예상 평점: 10
영화 ID: 10061, 예상 평점: 10
영화 ID: 100825, 예상 평점: 10
영화 ID: 10092, 예상 평점: 10
영화 ID: 10102, 예상 평점: 10
영화 ID: 10105, 예상 평점: 10
영화 ID: 10110, 예상 평점: 10


lightFM

###### 컨텐츠 기반 + 협업 필터링